In [1]:
import gzip
import math
import pickle
import zlib
import io
from collections import defaultdict
import pandas as pd
import numpy as np

# import scipy.stats

from sklearn.preprocessing import LabelEncoder

import engines
from utils import *

np.random.seed(2016)

transformers = {}

In [2]:
train_cols = ['customerID', 'Gender', 'State', 'transactionDate','product_code']

In [3]:
df_train = pd.read_csv('products.csv',usecols=train_cols)

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
## fill missing values
#products['customerID'].fillna('BBID_0000', inplace=True)
#df_train['promotion_description'].fillna('no_promo', inplace=True)
df_train['Gender'].fillna('no_gender', inplace=True)
df_train['State'].fillna('no_state', inplace=True)
#df_train['PinCode'].fillna(-1, inplace=True)
#df_train['DOB'].fillna("1", inplace=True)

In [6]:
df_train = df_train[np.isfinite(df_train['product_code'])]

In [7]:
cnt_srs = df_train['product_code'].astype('int64').value_counts().reset_index().head(1000)
#cnt_srs = products['product_code'].value_counts().reset_index()
cnt_srs.columns = ['product_code', 'frequency_count']
cnt_srs


,product_code,frequency_count
0,300776411,60994
1,300776410,58437
2,108037568,51338
3,300785148,41500
4,108100382,35259
5,300785150,34794
6,300840018,31149
7,108100362,27963
8,108005676,27727
9,300111517,25360


In [8]:
df_train = df_train[df_train['product_code'].isin(cnt_srs.product_code)].reset_index(drop=True)

df_train.head()

,customerID,Gender,State,transactionDate,product_code
0,BBID_2041,male,no_state,2016-01-19,3.007851e+08
1,BBID_2041,male,no_state,2016-03-19,3.009301e+08
2,BBID_2041,male,no_state,2016-01-19,1.000010e+09
3,BBID_2041,male,no_state,2016-05-01,1.081003e+08
4,BBID_2041,male,no_state,2016-03-20,1.081003e+08


In [9]:
df_train = pd.get_dummies(df_train, columns=['product_code'],prefix='_')

df_train.columns=df_train.columns.str.replace('_','')
df_train.reset_index(drop=True)
df_train.head()



,customerID,Gender,State,transactionDate,100105102.0,100105505.0,102064570.0,108000221.0,108000222.0,108000341.0,108000403.0,108000430.0,108000439.0,108000443.0,108000456.0,108000463.0,108000464.0,108000465.0,108000534.0,108000537.0,108000544.0,108000547.0,108000549.0,108000567.0,108000568.0,108000569.0,108000571.0,108000586.0,108000588.0,108000589.0,108000590.0,108000593.0,108000599.0,108000604.0,108000605.0,108000611.0,108000614.0,108000621.0,108000628.0,108000633.0,108000636.0,108000643.0,108000653.0,108000655.0,108000656.0,108000659.0,108000666.0,108000667.0,108000677.0,108000678.0,108000684.0,108000688.0,108000704.0,108000705.0,108000706.0,108000707.0,108000722.0,108000763.0,108000764.0,108000767.0,108000770.0,108000793.0,108000850.0,108000899.0,108000926.0,108000943.0,108000959.0,108000960.0,108000969.0,108000980.0,108000982.0,108000988.0,108001018.0,108001038.0,108001049.0,108001050.0,108001065.0,108001066.0,108001107.0,108001109.0,108001125.0,108001127.0,108001129.0,108001138.0,108001140.0,108001153.0,108001161.0,108001163.0,108001167.0,108001179.0,108001180.0,108001187.0,108001189.0,108001199.0,108001261.0,108001274.0,108001286.0,108001293.0,108001297.0,108001298.0,108002056.0,108002061.0,108002073.0,108002297.0,108002313.0,108002326.0,108003214.0,108003216.0,108003448.0,108003449.0,108003451.0,108003545.0,108003585.0,108003587.0,108003624.0,108003638.0,108003725.0,108003897.0,108003898.0,108003952.0,108004035.0,108004055.0,108004100.0,108004163.0,108004179.0,108004181.0,108004398.0,108004399.0,108004423.0,108004505.0,108004507.0,108004511.0,108004516.0,108004526.0,108004528.0,108004561.0,108004563.0,108004590.0,108004622.0,108004629.0,108004632.0,108004633.0,108004634.0,108004636.0,108004880.0,108004975.0,108004977.0,108005183.0,108005676.0,108005681.0,108005943.0,108007752.0,108007928.0,108008121.0,108008172.0,108008174.0,108008225.0,108008232.0,108008262.0,108008288.0,108008289.0,108008595.0,108008596.0,108008866.0,108008867.0,108009316.0,108009318.0,108010135.0,108010224.0,108010225.0,108010550.0,108010683.0,108012832.0,108013102.0,108013537.0,108013791.0,108013795.0,108013796.0,108013807.0,108014055.0,108014204.0,108014207.0,108014378.0,108014379.0,108014380.0,108014459.0,108015124.0,108015125.0,108015133.0,108015135.0,108015137.0,108015365.0,108015369.0,108015910.0,108016183.0,108016185.0,108016186.0,108016189.0,108016191.0,108016194.0,108017242.0,108017243.0,108017244.0,108017245.0,108017247.0,108017697.0,108017872.0,108017906.0,108018159.0,108018173.0,108018302.0,108018305.0,108018314.0,108018729.0,108018730.0,108018741.0,108018749.0,108018750.0,108018766.0,108018870.0,108018876.0,108018905.0,108018924.0,108018925.0,108019092.0,108019573.0,108020340.0,108020341.0,108020344.0,108020365.0,108020370.0,108020371.0,108020385.0,108020387.0,108020392.0,108020412.0,108020413.0,108020414.0,108020415.0,108020416.0,108020417.0,108020418.0,108020419.0,108020428.0,108020429.0,108020457.0,108020460.0,108020461.0,108020474.0,108020477.0,108020512.0,108020515.0,108020516.0,108020517.0,108020526.0,108020532.0,108020545.0,108020590.0,108020718.0,108020732.0,108021490.0,108021661.0,108022031.0,108022033.0,108022578.0,108022591.0,108022595.0,108022599.0,108023043.0,108023044.0,108023134.0,108023933.0,108024347.0,108024679.0,108024681.0,108024692.0,108024694.0,108024758.0,108025239.0,108026135.0,108026386.0,108026387.0,108026389.0,108026426.0,108026485.0,108026487.0,108026493.0,108026497.0,108026801.0,108026807.0,108026808.0,108026809.0,108026810.0,108026986.0,108026996.0,108026997.0,108027000.0,108027005.0,108027006.0,108027051.0,108027348.0,108027349.0,108027409.0,108027464.0,108027465.0,108027466.0,108027974.0,108028269.0,108028666.0,108028801.0,108029256.0,108029257.0,108029258.0,108029299.0,108029333.0,108029618.0,108029656.0,108029658.0,108029660.0,108029794.0,108030063.0,108030064.0,108030066.0,108030090.0,108030093.0,108030095.0,108030108.0,108030113.0,108030315.0,108031981.0,108031986.0,108032047.0,108032048.0,108032050.0

In [10]:
state_dict = {'MADHY PRADESH':'MADHYA PRADESH', 'TAMILNADU':'TAMIL NADU', 'MADHYA  PRADESH':'MADHYA PRADESH', 'HARAYANA':'HARYANA',
             'Jharkhand':'JHARKHAND','Tamilnadu':'TAMIL NADU','Tamil Nadu':'TAMIL NADU','Madhya Pradesh':'MADHYA PRADESH',
             'REST OF WEST BENGAL':'WEST BENGAL', 'west bengal':'WEST BENGAL','Uttar Pradesh':'UTTAR PRADESH', 'Delhi':'DELHI',
             'Bhopal':'BHOPAL','CHHATISGARH':'CHHATTISGARH','CHATTISGARH':'CHHATTISGARH', 'jharkhand':'JHARKHAND','Chandigarh':'CHANDIGARH',
             'UTTAR PRADESH WEST': 'UTTAR PRADESH','ODISHA':'ORISSA','MAHARASTRA':'MAHARASHTRA','madhya pradesh':'MADHYA PRADESH',
             'KARNATAK':'KARNATAKA','JAMMU and KASHMIR':'JAMMU AND KASHMIR','JAMMU KASHMIR':'JAMMU AND KASHMIR','Rajasthan':'RAJASTHAN',
             'east singhbhum':'JHARKHAND', 'ORRISA':'ORISSA','Andhra Pradesh':'ANDHRA PRADESH', 'UTTARANCHAL':'UTTARAKHAND',
             'Uttar pradesh':'UTTAR PRADESH','Maharashtra':'MAHARASHTRA','MP':'MADHYA PRADESH', 'UTTAR PRADESH EAST':'UTTAR PRADESH',
             'Punjab':'PUNJAB','maharashtra':'MAHARASHTRA','Karnataka':'KARNATAKA','M.P.':'MADHYA PRADESH','DAMAN':'DAMAN AND DIU',
             'HUBLI':'KARNATAKA','Tamil nadu':'TAMIL NADU','GUJRAT':'GUJARAT', 'Mp':'MADHYA PRADESH','Madhya pradesh':'MADHYA PRADESH',
             'West Bengal':'WEST BENGAL','Gujarat':'GUJARAT','UP':'UTTAR PRADESH','Chennai':'CHENNAI', 'm.p.':'MADHYA PRADESH',
             'kerala':'KERALA'}

df_train.replace({"State": state_dict}, inplace=True)

In [11]:
#df_train_april = df_train[df_train['transactionDate'] == '2016']
df_train_may = df_train[df_train['transactionDate']=='2016-05-28'] #made these two 2017 from 2016
df_train_june = df_train[df_train['transactionDate']=='2016-06-28']

df_train.drop('transactionDate', axis=1, inplace=True)

In [12]:
dfm = pd.merge(df_train_june,df_train_may, how='left', on=['customerID'], suffixes=('', '_prev'))

In [13]:
dfm.head()

customerID Gender     State transactionDate  100105102.0  100105505.0  \
0  BBID_204735   male  no_state      2016-06-28            0            0   
1  BBID_204735   male  no_state      2016-06-28            0            0   
2  BBID_204735   male  no_state      2016-06-28            0            0   
3  BBID_204735   male  no_state      2016-06-28            0            0   
4  BBID_204735   male  no_state      2016-06-28            0            0   

   102064570.0  108000221.0  108000222.0  108000341.0  108000403.0  \
0            0            0            0            0            0   
1            0            0            0            0            0   
2            0            0            0            0            0   
3            0            0            0            0            0   
4            0            0            0            0            0   

   108000430.0  108000439.0  108000443.0  108000456.0  108000463.0  \
0            0            0            0            0            0   
1            0            0            0            0            0   
2            0            0            0            0            0   
3            0            0            0            0            0   
4            0            0            0            0            0   

   108000464.0  108000465.0  108000534.0  108000537.0  108000544.0  \
0            0            0            0            0            0   
1            0            0            0            0            0   
2            0            0            0            0            0   
3            0            0            0            0            0   
4            0            0            0            0            0   

   108000547.0  108000549.0  108000567.0  108000568.0  108000569.0  \
0            0            0            0            0            0   
1            0            0            0            0            0   
2            0            0            0            0            0   
3            0            0            0            0            0   
4            0            0            0            0            0   

   108000571.0  108000586.0  108000588.0  108000589.0  108000590.0  \
0            0            0            0            0            0   
1            0            0            0            0            0   
2            0            0            0            0            0   
3            0            0            0            0            0   
4            0            0            0            0            0   

   108000593.0  108000599.0  108000604.0  108000605.0  108000611.0  \
0            0            0            0            0            0   
1            0            0            0            0            0   
2            0            0            0            0            0   
3            0            0            0            0            0   
4            0            0            0            0            0   

   108000614.0  108000621.0  108000628.0  108000633.0  108000636.0  \
0            0            0            0            0            0   
1            0            0            0            0            0   
2            0            0            0            0            0   
3            0            0            0            0            0   
4            0            0            0            0            0   

   108000643.0  108000653.0  108000655.0  108000656.0  108000659.0  \
0            0            0            0            0            0   
1            0            0            0            0            0   
2            0            0            0            0            0   
3            0            0            0            0            0   
4            0            0            0            0            0   

   108000666.0  108000667.0  108000677.0  108000678.0  108000684.0  \
0            0            0            0            0            0   
1            0            0      

In [14]:
dfm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3399 entries, 0 to 3398
Columns: 2007 entries, customerID to 300275441002.0_prev
dtypes: float64(1000), object(7), uint8(1000)
memory usage: 29.4+ MB


In [15]:
prevcols = [col for col in dfm.columns if '.0_prev' in col]
currcols = [col for col in dfm.columns if '.0' in col and '.0_prev' not in col]

In [16]:
for col in prevcols:
    dfm[col].fillna(0, inplace=True)

In [17]:
for col in currcols:
    dfm[col].fillna(0, inplace=True)

In [18]:
for col in currcols:
    dfm[col] = dfm[col] - dfm[col+'_prev']
    dfm[col] = dfm[col].apply(lambda x: max(x,0))

In [19]:
prevcols2 = [col for col in dfm.columns if '_prev' in col and col not in prevcols]
dfm.drop(prevcols2, axis=1, inplace=True)

In [20]:
dfm = dfm[dfm[currcols].sum(axis=1) >0]

In [21]:
dfm[currcols].sum().sum()

3394.0

In [22]:
dfm.head()

customerID Gender     State transactionDate  100105102.0  100105505.0  \
0  BBID_204735   male  no_state      2016-06-28          0.0          0.0   
1  BBID_204735   male  no_state      2016-06-28          0.0          0.0   
2  BBID_204735   male  no_state      2016-06-28          0.0          0.0   
3  BBID_204735   male  no_state      2016-06-28          0.0          0.0   
4  BBID_204735   male  no_state      2016-06-28          0.0          0.0   

   102064570.0  108000221.0  108000222.0  108000341.0  108000403.0  \
0          0.0          0.0          0.0          0.0          0.0   
1          0.0          0.0          0.0          0.0          0.0   
2          0.0          0.0          0.0          0.0          0.0   
3          0.0          0.0          0.0          0.0          0.0   
4          0.0          0.0          0.0          0.0          0.0   

   108000430.0  108000439.0  108000443.0  108000456.0  108000463.0  \
0          0.0          0.0          0.0          0.0          0.0   
1          0.0          0.0          0.0          0.0          0.0   
2          0.0          0.0          0.0          0.0          0.0   
3          0.0          0.0          0.0          0.0          0.0   
4          0.0          0.0          0.0          0.0          0.0   

   108000464.0  108000465.0  108000534.0  108000537.0  108000544.0  \
0          0.0          0.0          0.0          0.0          0.0   
1          0.0          0.0          0.0          0.0          0.0   
2          0.0          0.0          0.0          0.0          0.0   
3          0.0          0.0          0.0          0.0          0.0   
4          0.0          0.0          0.0          0.0          0.0   

   108000547.0  108000549.0  108000567.0  108000568.0  108000569.0  \
0          0.0          0.0          0.0          0.0          0.0   
1          0.0          0.0          0.0          0.0          0.0   
2          0.0          0.0          0.0          0.0          0.0   
3          0.0          0.0          0.0          0.0          0.0   
4          0.0          0.0          0.0          0.0          0.0   

   108000571.0  108000586.0  108000588.0  108000589.0  108000590.0  \
0          0.0          0.0          0.0          0.0          0.0   
1          0.0          0.0          0.0          0.0          0.0   
2          0.0          0.0          0.0          0.0          0.0   
3          0.0          0.0          0.0          0.0          0.0   
4          0.0          0.0          0.0          0.0          0.0   

   108000593.0  108000599.0  108000604.0  108000605.0  108000611.0  \
0          0.0          0.0          0.0          0.0          0.0   
1          0.0          0.0          0.0          0.0          0.0   
2          0.0          0.0          0.0          0.0          0.0   
3          0.0          0.0          0.0          0.0          0.0   
4          0.0          0.0          0.0          0.0          0.0   

   108000614.0  108000621.0  108000628.0  108000633.0  108000636.0  \
0          0.0          0.0          0.0          0.0          0.0   
1          0.0          0.0          0.0          0.0          0.0   
2          0.0          0.0          0.0          0.0          0.0   
3          0.0          0.0          0.0          0.0          0.0   
4          0.0          0.0          0.0          0.0          0.0   

   108000643.0  108000653.0  108000655.0  108000656.0  108000659.0  \
0          0.0          0.0          0.0          0.0          0.0   
1          0.0          0.0          0.0          0.0          0.0   
2          0.0          0.0          0.0          0.0          0.0   
3          0.0          0.0          0.0          0.0          0.0   
4          0.0          0.0          0.0          0.0          0.0   

   108000666.0  108000667.0  108000677.0  108000678.0  108000684.0  \
0          0.0          0.0          0.0          0.0          0.0   
1          0.0          0.0      

In [23]:
df_new = pd.DataFrame()

for index, row in dfm.iterrows():
    if index%1000 ==0:
        print(index)
    for i,col in enumerate(currcols):
        if row[col] == 1:
            row['target'] = currcols.index(col)
            df_new = df_new.append(row)
            
df_new.drop(currcols, axis=1, inplace=True)

0
1000
2000
3000


In [24]:
df_new.head()

,1000000263.0_prev,1000001687.0_prev,1000001696.0_prev,1000001721.0_prev,1000001730.0_prev,1000001767.0_prev,1000005980.0_prev,1000009993.0_prev,1000010375.0_prev,1000010393.0_prev,1000010403.0_prev,1000010411.0_prev,1000014043.0_prev,1000016457.0_prev,1000016711.0_prev,1000017733.0_prev,1000019395.0_prev,1000019401.0_prev,1000019459.0_prev,1000019571.0_prev,1000025152.0_prev,1000025840.0_prev,1000025937.0_prev,1000026628.0_prev,1000026742.0_prev,1000026751.0_prev,1000027208.0_prev,1000027846.0_prev,1000031858.0_prev,1000031931.0_prev,1000032404.0_prev,1000033916.0_prev,1000034055.0_prev,1000034447.0_prev,1000038522.0_prev,1000049681.0_prev,1000051274.0_prev,1000052897.0_prev,1000053898.0_prev,1000053932.0_prev,1000058092.0_prev,1000059431.0_prev,1000060132.0_prev,1000064599.0_prev,1000067633.0_prev,1000067672.0_prev,1000067678.0_prev,1000069534.0_prev,1000071425.0_prev,1000071500.0_prev,1000077019.0_prev,1000077077.0_prev,1000081409.0_prev,1000081523.0_prev,1000082341.0_prev,1000088409.0_prev,1000088419.0_prev,1000088454.0_prev,1000088464.0_prev,1000088467.0_prev,1000088473.0_prev,1000088477.0_prev,1000088494.0_prev,1000088500.0_prev,1000088510.0_prev,1000088540.0_prev,1000091199.0_prev,1000091208.0_prev,1000092917.0_prev,1000093979.0_prev,1000094761.0_prev,1000095289.0_prev,1000096658.0_prev,1000096680.0_prev,1000110589.0_prev,1000110676.0_prev,1000116549.0_prev,1000117286.0_prev,1000117635.0_prev,1000123958.0_prev,1000125074.0_prev,1000129459.0_prev,1000129487.0_prev,1000129541.0_prev,1000130998.0_prev,1000131053.0_prev,1000139414.0_prev,1000146094.0_prev,1000152699.0_prev,1000155818.0_prev,1000155826.0_prev,1000158186.0_prev,1000163880.0_prev,1000167930.0_prev,1000170737.0_prev,1000171548.0_prev,1000172148.0_prev,1000172186.0_prev,1000190676.0_prev,1000196248.0_prev,1000197484.0_prev,1000197561.0_prev,1000203181.0_prev,1000203496.0_prev,1000203533.0_prev,1000206088.0_prev,1000208501.0_prev,1000209360.0_prev,1000209934.0_prev,1000214513.0_prev,1000214541.0_prev,1000214550.0_prev,1000214904.0_prev,1000215039.0_prev,1000230660.0_prev,1000257306.0_prev,1000257325.0_prev,1000268511.0_prev,1000269684.0_prev,1000269704.0_prev,1000310765.0_prev,1000323996.0_prev,1000327894.0_prev,1000334998.0_prev,1000336252.0_prev,1000349544.0_prev,1000368615.0_prev,1000434939.0_prev,1000443203.0_prev,1000443242.0_prev,1000443270.0_prev,1000443280.0_prev,1000446297.0_prev,1000446386.0_prev,1000446401.0_prev,1000446492.0_prev,1000446500.0_prev,1000446546.0_prev,1000450352.0_prev,1000470646.0_prev,1000475623.0_prev,1000482559.0_prev,1000486459.0_prev,1000486566.0_prev,1000486622.0_prev,1000486722.0_prev,1000491251.0_prev,1000492222.0_prev,1000492241.0_prev,1000494165.0_prev,1000494206.0_prev,1000494275.0_prev,1000501584.0_prev,1000503970.0_prev,1000516470.0_prev,1000541095.0_prev,1000542693.0_prev,1000551642.0_prev,1000576685.0_prev,1000577194.0_prev,1000578952.0_prev,1000579478.0_prev,1000582493.0_prev,1000582495.0_prev,1000582496.0_prev,1000591804.0_prev,1000591805.0_prev,1000595250.0_prev,1000602688.0_prev,1000602770.0_prev,1000606564.0_prev,1000609658.0_prev,1000615919.0_prev,1000619267.0_prev,1000619269.0_prev,1000638371.0_prev,1000638372.0_prev,1000638373.0_prev,1000652689.0_prev,1000652760.0_prev,1000652761.0_prev,1000694851.0_prev,1000776349.0_prev,100105102.0_prev,100105505.0_prev,102064570.0_prev,108000221.0_prev,108000222.0_prev,108000341.0_prev,108000403.0_prev,108000430.0_prev,108000439.0_prev,108000443.0_prev,108000456.0_prev,108000463.0_prev,108000464.0_prev,108000465.0_prev,108000534.0_prev,108000537.0_prev,108000544.0_prev,108000547.0_prev,108000549.0_prev,108000567.0_prev,108000568.0_prev,108000569.0_prev,108000571.0_prev,108000586.0_prev,108000588.0_prev,108000589.0_prev,108000590.0_prev,108000593.0_prev,108000599.0_prev,108000604.0_prev,108000605.0_prev,108000611.0_prev,108000614.0_prev,108000621.0_prev,108000628.0_prev,108000633.0_prev,108000636.0_prev,108000643.0_prev,108000653.0_prev,108000655.0_prev,108000656.0_prev,10800065

In [25]:
df_new.to_csv('cleaned_data/final_data_prepared.csv')

In [ ]:
df_new.columns